Import Libraries

In [1]:
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor 

Download MNIST Data

In [2]:
train = datasets.MNIST(root="Data", download=True, train=True, transform=ToTensor())
print(type(train))
dataset = DataLoader(train, batch_size=32)

<class 'torchvision.datasets.mnist.MNIST'>


Create the Class for the model

In [3]:
class ImageClassifier(nn.Module):
    # Initialize the model
    def __init__(self):
        super().__init__() 

        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3, 3)),
            nn.ReLU(),

            nn.Conv2d(32, 64, (3, 3)),
            nn.ReLU(),

            nn.Conv2d(64, 64, (3, 3)),
            nn.ReLU(),

            nn.Flatten(),
            nn.Linear(64 * (28-6) * (28-6), 10)
        )
    
    # Forward pass
    def forward(self, x):
        return self.model(x)

Create a model

In [4]:
classifier = ImageClassifier().to("cuda")
optimizer = Adam(classifier.parameters(), lr=1e-3)
lossFn = nn.CrossEntropyLoss()

if __name__ == "__main__":
    for epoch in range(10):
        for batch in dataset:
            X, y = batch
            X, y = X.to("cuda"), y.to("cuda")

            y_pred = classifier(X)
            loss = lossFn(y_pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")
    
    with open("model_state.pt", "wb") as file:
        save(classifier.state_dict(), file)

Epoch: 0, Loss: 0.022236613556742668
Epoch: 1, Loss: 0.0037643457762897015
Epoch: 2, Loss: 0.0018432220676913857
Epoch: 3, Loss: 9.976848377846181e-05
Epoch: 4, Loss: 0.00019332062220200896
Epoch: 5, Loss: 7.401911716442555e-06
Epoch: 6, Loss: 6.373724772856804e-06
Epoch: 7, Loss: 6.146453870314872e-06
Epoch: 8, Loss: 3.2185419058805564e-06
Epoch: 9, Loss: 2.506354940123856e-05
